In [1]:
%pip install opencv-python numpy kagglehub

usage: conda [-h] [-V] command ...

conda is a tool for managing and deploying applications, environments and packages.

Options:

positional arguments:
  command
    clean        Remove unused packages and caches.
    compare      Compare packages between conda environments.
    config       Modify configuration values in .condarc. This is modeled
                 after the git config command. Writes to the user .condarc
                 file (/aul/homes/dvahi001/.condarc) by default.
    create       Create a new conda environment from a list of specified
                 packages.
    help         Displays a list of available conda commands and their help
                 strings.
    info         Display information about current conda install.
    init         Initialize conda for shell interaction. [Experimental]
    install      Installs a list of packages into a specified conda
                 environment.
    list         List linked packages in a conda environment.
    packa

In [15]:
import os
import cv2
import numpy as np
from PIL import Image as PILImage
from IPython.display import display


PATH = "./processed_images/"

def resize_image(image, target_size=(224, 224)):
    # If the image is grayscale, convert it to 3 channels (RGB)
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    h, w = image.shape[:2]
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w = int(w * scale)
    new_h = int(h * scale)

    resized_image = cv2.resize(image, (new_w, new_h))
    canvas = np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)
    top = (target_size[0] - new_h) // 2
    left = (target_size[1] - new_w) // 2
    canvas[top:top + new_h, left:left + new_w] = resized_image

    return canvas



def set_black_background(image, threshold=20):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_image, threshold, 255, cv2.THRESH_BINARY_INV)
    image[mask == 255] = [0, 0, 0]
    return image


def preprocess_image(image, grayscale=True, target_size=(224, 224)):
    image = resize_image(image, target_size)
    image = set_black_background(image)
    if grayscale:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image


def random_rotation(image):
    angle = np.random.uniform(-30, 30)
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, M, (w, h))

def random_flip(image):
    if np.random.rand() > 0.5:
        return cv2.flip(image, 1)  
    return image

def random_brightness(image):
    factor = np.random.uniform(0.5, 1.5)
    return cv2.convertScaleAbs(image, alpha=factor, beta=0)


def random_zoom(image):
    h, w = image.shape[:2]
    zoom = np.random.uniform(0.3, 2.0)  
    new_h, new_w = int(h * zoom), int(w * zoom)
    image = cv2.resize(image, (new_w, new_h))
    if new_h < h or new_w < w:
        top = (h - new_h) // 2
        left = (w - new_w) // 2
        cropped = image[top:top + new_h, left:left + new_w]
    else:
        cropped = image[:h, :w]
    return resize_image(cropped)


def save_processed_images(image_files, batch_size=50, output_dir="./augmented_photos"):
    os.makedirs(output_dir, exist_ok=True)    
    for i, image_file in enumerate(image_files):
        image_path = os.path.join(PATH, image_file)
        print(f"Processing image: {image_path}")

        # Load the image
        image = cv2.imread(image_path)

        # Check if the image is loaded correctly
        if image is None:
            print(f"Error: Failed to load image {image_path}")
            continue  # Skip this image and move to the next one

        processed_image = preprocess_image(image)

        # Apply augmentations and save each one
        try:
            rotation_image = random_rotation(processed_image)
            rotation_path = os.path.join(output_dir, f"rotated_{i + 1}.jpg")
            PILImage.fromarray(cv2.cvtColor(rotation_image, cv2.COLOR_BGR2RGB)).save(rotation_path)

            flip_image = random_flip(processed_image)
            flip_path = os.path.join(output_dir, f"flipped_{i + 1}.jpg")
            PILImage.fromarray(cv2.cvtColor(flip_image, cv2.COLOR_BGR2RGB)).save(flip_path)

            brightness_image = random_brightness(processed_image)
            brightness_path = os.path.join(output_dir, f"brightness_{i + 1}.jpg")
            PILImage.fromarray(cv2.cvtColor(brightness_image, cv2.COLOR_BGR2RGB)).save(brightness_path)

            zoom_image = random_zoom(processed_image)
            zoom_path = os.path.join(output_dir, f"zoomed_{i + 1}.jpg")
            PILImage.fromarray(cv2.cvtColor(zoom_image, cv2.COLOR_BGR2RGB)).save(zoom_path)

            print(f"Processed images for {image_file} saved to {output_dir}")
        except Exception as e:
            print(f"Error processing image {image_file}: {e}")


In [16]:
image_files = [f for f in os.listdir(PATH) if f.endswith(('.png', '.jpg', '.jpeg'))]
save_processed_images(image_files)

Processing image: ./processed_images/fear_449.jpg
Processed images for fear_449.jpg saved to ./augmented_photos
Processing image: ./processed_images/fear_498.jpg
Processed images for fear_498.jpg saved to ./augmented_photos
Processing image: ./processed_images/happy_30.jpg
Processed images for happy_30.jpg saved to ./augmented_photos
Processing image: ./processed_images/fear_511.jpg
Processed images for fear_511.jpg saved to ./augmented_photos
Processing image: ./processed_images/happy_110.jpg
Processed images for happy_110.jpg saved to ./augmented_photos
Processing image: ./processed_images/fear_435.jpg
Processed images for fear_435.jpg saved to ./augmented_photos
Processing image: ./processed_images/angry_360.jpg
Processed images for angry_360.jpg saved to ./augmented_photos
Processing image: ./processed_images/angry_416.jpg
Processed images for angry_416.jpg saved to ./augmented_photos
Processing image: ./processed_images/sad_216.jpg
Processed images for sad_216.jpg saved to ./augme